In [12]:
import datetime
import pandas as pd
import numpy as np
import sklearn.preprocessing as pp #pour les scaling
from sklearn.cross_validation import train_test_split

In [13]:
df = pd.read_csv("DonneesMeteo.txt",delim_whitespace=True, low_memory=False, skiprows=1)
n=len(df)

In [14]:
#Version arrondie à la minute de la ligne du dessous : num = pd.to_datetime(pd.Series.round(df['hh(UTC)']*3600)*1000000000)
num = pd.to_datetime(df['hh(UTC)']*3600*1000000000)
indext = pd.DatetimeIndex(num)
indexd = pd.DatetimeIndex(pd.to_datetime(df['Date'], format="%Y%m%d"))

df['Day'] = indexd.dayofyear

In [15]:
#Faire une timestamp
'''ts = []
i=0
while (i<len(df)) :
    ts.append(datetime.datetime.combine(datetime.date(indexd[i].year, indexd[i].month, indexd[i].day), datetime.time(indext[i].hour, indext[i].minute, indext[i].second)))
    i = i + 1
df['TimeStamp'] = ts'''

"ts = []\ni=0\nwhile (i<len(df)) :\n    ts.append(datetime.datetime.combine(datetime.date(indexd[i].year, indexd[i].month, indexd[i].day), datetime.time(indext[i].hour, indext[i].minute, indext[i].second)))\n    i = i + 1\ndf['TimeStamp'] = ts"

On va à présent se concentrer sur la prédiction de :
    - puissance
Selon
    - T
    - vent
    - skycover
    - precipitation

In [22]:

df.columns.values.tolist()

['Date',
 'hh(UTC)',
 'P1(W)',
 'Tp1(C)',
 'P2(W)',
 'Tp2(C)',
 'IrrPOA(W/m2)',
 'Gz2(W/m2)',
 'Drz2(W/m2)',
 'Dfz2(W/m2)',
 'IRz2(W/m2)',
 'T(C)',
 'WS(m/s)',
 'WD(deg)',
 'mpGz1(W/m2)',
 'IRz1dn(W/m2)',
 'IRz1up(W/m2)',
 'mpT(C)',
 'mpWS(m/s)',
 'mpWD(deg)',
 'SZA(deg)',
 'SAA(deg)',
 'Day']

# RandomForest

In [23]:
#Paramètres
pTrain = 0.6
pTest = 0.4

colsX = ['Day',
    'Date',
 'hh(UTC)',
 'Tp1(C)',
 'Tp2(C)',
 'IrrPOA(W/m2)',
 'Gz2(W/m2)',
 'Drz2(W/m2)',
 'Dfz2(W/m2)',
 'IRz2(W/m2)',
 'T(C)',
 'WS(m/s)',
 'WD(deg)',
 'mpGz1(W/m2)',
 'IRz1dn(W/m2)',
 'IRz1up(W/m2)',
 'mpT(C)',
 'mpWS(m/s)',
 'mpWD(deg)',
 'SZA(deg)',
 'SAA(deg)',]
#colsX = ["DayPart", "T(C)", ]
colsY = ["P1(W)", "P2(W)"]
cols = colsX + colsY
#print(cols)

In [32]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model

# Load the dataset
Z = np.array(df[cols]).astype(np.float)

#Remove nan and infinite values
masknan = ~np.any(np.isnan(Z), axis=1)
Z = Z[masknan]
maskfin = np.any(np.isfinite(Z), axis=1)
Z = Z[maskfin]
#feature scaling
Z = pp.StandardScaler().fit_transform(Z)

#Split the treated data between X and y
deb = 0
sep = len(colsX)#separteur entre les X et les Y
fin = sep+len(colsY)
X=Z[:,deb:sep]
y=Z[:,sep:fin]

# Split the data into training/testing sets
#X = np.reshape(X,(len(X),len(colsX)))#pour "transposer"
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=pTrain, test_size=pTest )
#feature scaling sur le train set !

In [34]:
X_test.shape

(3231, 21)

In [35]:
# Create linear regression object
from sklearn import ensemble
regr = sklearn.ensemble.RandomForestRegressor()
# Train the model using the training sets
regr.fit(X_train, y_train)
# The mean square error
print("Residual sum of squares: %.2f" % np.mean((regr.predict(X_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

plotterIterations(X_test,y_test,2,1,0)

Residual sum of squares: 0.04
Variance score: 0.96


In [28]:
# Plot outputs 2D
import matplotlib.cm as cm

def plotterIterations(X,y,nX,ny,nIt) :
    #On se ramène à des journées
    EltIteration = X[:,nIt]
    Iterations = np.unique(EltIteration)

    #Gérer plusieurs couleurs
    colors = cm.rainbow(np.linspace(0, 1, len(Iterations)))

    for k,c in zip(Iterations,colors)  :
        #On prend que le jour
        condition = EltIteration == k #entre 60 et 153
        xPlot = np.extract(condition, X[:,nX])
        yPlot = np.extract(condition, y[:,ny])
        plt.plot(xPlot, yPlot, color=c,linewidth=2)

    #plt.plot(X_test, regr.predict(X_test), color='blue',linewidth=3)
    plt.xticks(())
    plt.yticks(())

    #plt.xlabel(colsX[nX])
    #plt.ylabel(colsY[nY])
    plt.title('')

    plt.show()